# Text to Text Explanation: Open Ended Text Generation Using GPT2

This notebook demonstrates use of generating model explanations for open ended text generation using gpt2. In this demo, we use the pretrained gpt2 model provided by hugging face (https://huggingface.co/gpt2) to explain the model used to generate text based on passing custom model generation configurations on an intial provided text.

In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import shap
import torch

### Load model and tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model =  AutoModelForCausalLM.from_pretrained("gpt2").cuda()

Below, we set certain model configurations. We need to define if the model is a decoder or encoder-decoder.
This can be set through the 'is_decoder' or 'is_encoder_decoder' param in model's config file.
We can also set custom model generation parameters which will be used during the output text generation decoding process.

In [3]:
# set model decoder to true and generation params
model.config.is_decoder=True
model.config.text_generation_params = {
    "do_sample": True,
    "max_length": 50,
    "temperature": 0.7,
    "top_k": 0
  }

### Define initial text

In [4]:
s=["I enjoy walking with my cute dog"]

### Create an explainer object

In [5]:
explainer = shap.Explainer(model,tokenizer)

explainers.Partition is still in an alpha state, so use with caution...


### Compute shap values

In [6]:
shap_values = explainer(s)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


### Visualize shap explanations

In [7]:
shap.plots.text(shap_values)

,I,enjoy,walking,with,my,cute,dog
and,-0.22,0.341,0.243,0.018,-0.384,1.097,1.073
watching,0.079,0.723,3.184,0.372,0.503,0.334,-0.201
the,0.228,-0.284,0.241,-0.085,0.055,-0.423,0.194
Heroes,0.534,-0.023,-0.16,0.085,-0.197,-0.358,-0.631
of,-0.023,0.385,-0.028,0.117,0.089,-0.071,0.1
the,0.187,0.108,0.124,0.05,-0.01,-0.02,0.064
Storm,0.004,-0.017,-0.092,0.078,-0.099,0.444,-0.049
stream,0.188,-0.187,0.348,-0.055,0.12,0.139,0.748
after,-0.242,-0.312,-0.111,0.128,0.106,0.155,-0.027
work,0.967,0.463,0.41,0.068,0.246,0.198,0.619


### Another example...

In [8]:
s=['Scientists confirmed the worst possible outcome: the massive asteroid will collide with Earth']

In [9]:
explainer = shap.Explainer(model,tokenizer)

explainers.Partition is still in an alpha state, so use with caution...


In [10]:
shap_values = explainer(s)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [11]:
shap.plots.text(shap_values)

,Scientists confirmed the worst,possible outcome,: the,massive,asteroid,will,collide,with,Earth
in,0.478,-0.183,-0.648,0.063,0.251,0.135,0.979,0.489,1.436
a,-0.178,0.295,-0.174,-0.132,-0.121,0.106,0.885,-0.168,0.31
massive,0.767,0.119,-0.191,0.655,0.688,0.222,2.157,0.214,0.009
collision,0.896,0.192,0.078,-0.053,1.062,0.443,3.414,0.001,0.413
that,0.693,-0.016,-0.039,-0.047,-0.09,-0.285,1.08,0.409,-0.912
will,-1.035,0.446,-0.263,0.197,0.338,1.81,0.622,0.068,0.662
cause,-0.247,0.087,0.106,-0.055,0.044,0.305,0.531,0.147,0.179
widespread,1.294,0.507,-0.143,-0.061,0.12,-0.146,0.009,-0.079,-0.016
devastation,0.151,-0.087,-0.219,0.365,0.59,-0.013,0.038,-0.048,0.251
.,-0.725,0.237,0.382,-0.097,-0.183,0.052,0.086,0.264,0.312


##  Custom text generation

Below we demonstrate how to explain the liklihood of generating a particular output sentence given an input sentence using the model. For this, we define an input-output sentence pair and a custom text generation function which returns the target sentence ids of the output sentence of choice to be generated by the model

In [12]:
sentence_pairs = {
    "I know many people who are Russian." : "They love their vodka!"
}

In [13]:
def generate_target_sentence_ids_for_output(x):
    # we get the target sentence by a dictionary lookup from the previously definned sentence pairs
    target_sentence = sentence_pairs[x]
    target_sentence_ids = torch.tensor([tokenizer.encode(target_sentence)])
    return target_sentence_ids

In [14]:
wrapped_model = shap.models.PTTeacherForcingLogits(model, tokenizer, generation_function_for_target_sentence_ids=generate_target_sentence_ids_for_output)

In [15]:
explainer = shap.Explainer(wrapped_model,tokenizer)

explainers.Partition is still in an alpha state, so use with caution...


In [16]:
s = list(sentence_pairs.keys())

In [17]:
shap_values = explainer(s)

In [18]:
shap.plots.text(shap_values)

,I,know,many,people,who,are,Russian,.
They,-1.954,-1.215,-1.509,-1.876,-0.569,-1.412,1.81,4.452
love,0.64,0.145,0.173,0.079,-0.569,0.202,0.252,0.104
their,0.014,-0.11,0.226,0.187,0.05,0.197,0.067,-0.307
vodka,-0.096,-0.118,-0.153,-0.034,0.028,0.108,2.477,-0.389
!,-0.099,-0.126,-0.164,-0.467,-0.298,0.06,0.032,-0.145
